In [0]:
import argparse
import json
import importlib
import glob
import os

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

#Mount gdrive for folder access

In [4]:
def gdrive_mount():
    from google.colab import drive
    drive.mount('/content/gdrive')

gdrive_mount()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Utilities


In [0]:
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)


def save_json(data, target_file):
    with open(target_file, 'w') as f:
        json.dump(data, f, indent=2, sort_keys=True)


def load_config(config_file):
    config = load_json(config_file)
    return config


def random_crop(img, crop_dims):
    h, w = img.shape[0], img.shape[1]
    ch, cw = crop_dims[0], crop_dims[1]
    assert h >= ch, 'image height is less than crop height'
    assert w >= cw, 'image width is less than crop width'
    x = np.random.randint(0, w - cw + 1)
    y = np.random.randint(0, h - ch + 1)
    return img[y:(y+ch), x:(x+cw), :]


def random_horizontal_flip(img):
    assert len(img.shape) == 3, 'input tensor must have 3 dimensions (height, width, channels)'
    assert img.shape[2] == 3, 'image not in channels last format'
    if np.random.random() < 0.5:
        img = img.swapaxes(1, 0)
        img = img[::-1, ...]
        img = img.swapaxes(0, 1)
    return img


def load_image(img_file, target_size):
    return np.asarray(tf.keras.preprocessing.image.load_img(img_file, target_size=target_size))


def normalize_labels(labels):
    labels_np = np.array(labels)
    return labels_np / labels_np.sum()


def calc_mean_score(score_dist):
    score_dist = normalize_labels(score_dist)
    return (score_dist*np.arange(1, 11)).sum()


def ensure_dir_exists(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

def ensure_file_exists(file):
  if not os.path.exists(file):
      os.touch(file)


def load_samples(samples_file):
    return load_json(samples_file)

#Loss function

In [0]:
def earth_movers_distance(y_true, y_pred):
    cdf_true = K.cumsum(y_true, axis=-1)
    cdf_pred = K.cumsum(y_pred, axis=-1)
    emd = K.sqrt(K.mean(K.square(cdf_true - cdf_pred), axis=-1))
    return K.mean(emd)

#Data Generator

In [0]:
class TrainDataGenerator(tf.keras.utils.Sequence):
    '''inherits from Keras Sequence base object, allows to use multiprocessing in .fit_generator'''
    def __init__(self, samples, img_dir, batch_size, n_classes, basenet_preprocess, img_format,
                 img_load_dims=(256, 256), img_crop_dims=(224, 224), shuffle=True):
        self.samples = samples
        self.img_dir = img_dir
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.basenet_preprocess = basenet_preprocess  # Keras basenet specific preprocessing function
        self.img_load_dims = img_load_dims  # dimensions that images get randomly cropped toload_dims  # dimensions that images get resized into when loaded
        self.img_crop_dims = img_crop_dims  # dimensions that images get randomly cropped to
        self.shuffle = shuffle
        self.img_format = img_format
        self.on_epoch_end()  # call ensures that samples are shuffled in first epoch if shuffle is set to True

    def __len__(self):
        return int(np.ceil(len(self.samples) / self.batch_size))  # number of batches per epoch

    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]  # get batch indexes
        batch_samples = [self.samples[i] for i in batch_indexes]  # get batch samples
        X, y = self.__data_generator(batch_samples)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.samples))
        if self.shuffle is True:
            np.random.shuffle(self.indexes)

    def __data_generator(self, batch_samples):
        # initialize images and labels tensors for faster processing
        X = np.empty((len(batch_samples), *self.img_crop_dims, 3))
        y = np.empty((len(batch_samples), self.n_classes))

        for i, sample in enumerate(batch_samples):
            # load and randomly augment image
            img_file = os.path.join(self.img_dir, '{}.{}'.format(sample['image_id'], self.img_format))
            img = load_image(img_file, self.img_load_dims)
            if img is not None:
                img = random_crop(img, self.img_crop_dims)
                img = random_horizontal_flip(img)
                X[i, ] = img

            # normalize labels
            y[i, ] = normalize_labels(sample['label'])

        # apply basenet specific preprocessing
        # input is 4D numpy array of RGB values within [0, 255]
        X = self.basenet_preprocess(X)

        return X, y

In [0]:
class TestDataGenerator(tf.keras.utils.Sequence):
    '''inherits from Keras Sequence base object, allows to use multiprocessing in .fit_generator'''
    def __init__(self, samples, img_dir, batch_size, n_classes, basenet_preprocess, img_format,
                 img_load_dims=(224, 224)):
        self.samples = samples
        self.img_dir = img_dir
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.basenet_preprocess = basenet_preprocess  # Keras basenet specific preprocessing function
        self.img_load_dims = img_load_dims  # dimensions that images get resized into when loaded
        self.img_format = img_format
        self.on_epoch_end()  # call ensures that samples are shuffled in first epoch if shuffle is set to True

    def __len__(self):
        return int(np.ceil(len(self.samples) / self.batch_size))  # number of batches per epoch

    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]  # get batch indexes
        batch_samples = [self.samples[i] for i in batch_indexes]  # get batch samples
        X, y = self.__data_generator(batch_samples)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.samples))

    def __data_generator(self, batch_samples):
        # initialize images and labels tensors for faster processing
        X = np.empty((len(batch_samples), *self.img_load_dims, 3))
        y = np.empty((len(batch_samples), self.n_classes))

        for i, sample in enumerate(batch_samples):
            # load and randomly augment image
            img_file = os.path.join(self.img_dir, '{}.{}'.format(sample['image_id'], self.img_format))
            img = load_image(img_file, self.img_load_dims)
            if img is not None:
                X[i, ] = img

            # normalize labels
            if sample.get('label') is not None:
                y[i, ] = normalize_labels(sample['label'])

        # apply basenet specific preprocessing
        # input is 4D numpy array of RGB values within [0, 255]
        X = self.basenet_preprocess(X)

        return X, y

#Model Builder


In [0]:
class Nima:
    def __init__(self, base_model_name, n_classes=10, learning_rate=0.001, dropout_rate=0, loss=earth_movers_distance,
                 decay=0, weights='imagenet'):
        self.n_classes = n_classes
        self.base_model_name = base_model_name
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.loss = loss
        self.decay = decay
        self.weights = weights
        self._get_base_module()

    def _get_base_module(self):
        # import Keras base model module
        if self.base_model_name == 'InceptionV3':
            self.base_module = importlib.import_module('tensorflow.keras.applications.inception_v3')
        elif self.base_model_name == 'InceptionResNetV2':
            self.base_module = importlib.import_module('tensorflow.keras.applications.inception_resnet_v2')
        else:
            self.base_module = importlib.import_module('tensorflow.keras.applications.'+self.base_model_name.lower())

    def build(self):
        # get base model class
        BaseCnn = getattr(self.base_module, self.base_model_name)

        # load pre-trained model
        self.base_model = BaseCnn(input_shape=(224, 224, 3), weights=self.weights, include_top=False, pooling='avg')

        # add dropout and dense layer
        x = Dropout(self.dropout_rate)(self.base_model.output)
        x = Dense(units=self.n_classes, activation='softmax')(x)

        self.nima_model = Model(self.base_model.inputs, x)

    def compile(self):
        self.nima_model.compile(optimizer=Adam(lr=self.learning_rate, decay=self.decay), loss=self.loss)

    def preprocessing_function(self):
        return self.base_module.preprocess_input

#Train

In [0]:
def train(
    base_model_name,
    n_classes,
    samples,
    image_dir,
    batch_size,
    epochs_train_dense,
    epochs_train_all,
    learning_rate_dense,
    learning_rate_all,
    dropout_rate,
    weights_dir,
    log_dir,
    img_format='jpg',
    existing_weights=None,
    multiprocessing_data_load=False,
    num_workers_data_load=2,
    decay_dense=0,
    decay_all=0,
    **kwargs
):

    # build NIMA model and load existing weights if they were provided in config
    nima = Nima(
        base_model_name, n_classes, learning_rate_dense, dropout_rate, decay=decay_dense
    )
    nima.build()

    if existing_weights is not None:
        nima.nima_model.load_weights(existing_weights)

    # split samples in train and validation set, and initialize data generators
    samples_train, samples_test = train_test_split(
        samples, test_size=0.05, shuffle=True, random_state=10207
    )

    training_generator = TrainDataGenerator(
        samples_train,
        image_dir,
        batch_size,
        n_classes,
        nima.preprocessing_function(),
        img_format=img_format,
    )

    validation_generator = TestDataGenerator(
        samples_test,
        image_dir,
        batch_size,
        n_classes,
        nima.preprocessing_function(),
        img_format=img_format,
    )

    # initialize callbacks TensorBoard and ModelCheckpoint
    tensorboard = TensorBoard(
        log_dir=log_dir, update_freq='batch'
    )

    model_save_name = (
        'weights_' + base_model_name.lower() + '_{epoch:02d}_{val_loss:.3f}.hdf5'
    )
    model_file_path = os.path.join(weights_dir, model_save_name)
    model_checkpointer = ModelCheckpoint(
        filepath=model_file_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        save_weights_only=True,
    )

    # start training only dense layers
    #freeze
    for layer in nima.base_model.layers:
        layer.trainable = False

    nima.compile()
    nima.nima_model.summary()

    nima.nima_model.fit_generator(
        generator=training_generator,
        validation_data=validation_generator,
        epochs=epochs_train_dense,
        verbose=1,
        use_multiprocessing=multiprocessing_data_load,
        workers=num_workers_data_load,
        max_queue_size=30,
        callbacks=[tensorboard, model_checkpointer],
    )

    # start training all layers
    for layer in nima.base_model.layers:
        layer.trainable = True

    nima.learning_rate = learning_rate_all
    nima.decay = decay_all
    nima.compile()
    nima.nima_model.summary()

    nima.nima_model.fit_generator(
        generator=training_generator,
        validation_data=validation_generator,
        epochs=epochs_train_dense + epochs_train_all,
        initial_epoch=epochs_train_dense,
        verbose=1,
        use_multiprocessing=multiprocessing_data_load,
        workers=num_workers_data_load,
        max_queue_size=30,
        callbacks=[tensorboard, model_checkpointer],
    )

    K.clear_session()


#Main for Train

In [13]:
IMAGE_DIR = '/content/gdrive/My Drive/TID2013/distorted_images'

WEIGHTS_DIR = '/content/gdrive/My Drive/GC-IQA_Prism/IQA_Prism/weights/E1'
LOG_DIR = '/content/gdrive/My Drive/GC-IQA_Prism/IQA_Prism/logs/E1'
ensure_dir_exists(WEIGHTS_DIR)
ensure_dir_exists(LOG_DIR)

CONFIG_DIR = '/content/gdrive/My Drive/GC-IQA_Prism/IQA_Prism/config'
CONFIG_FILE = os.path.join(CONFIG_DIR, 'config_technical_gpu.json')
config = load_config(CONFIG_FILE)

SAMPLES_DIR = '/content/gdrive/My Drive/GC-IQA_Prism/IQA_Prism/labels/TID2013'
SAMPLES_FILE = os.path.join(SAMPLES_DIR, 'tid_labels_train.json')
samples = load_samples(SAMPLES_FILE)

train(samples=samples, weights_dir=WEIGHTS_DIR, log_dir=LOG_DIR, image_dir=IMAGE_DIR, **config)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

Process Keras_worker_ForkPoolWorker-4:
Process Keras_worker_ForkPoolWorker-3:
Process Keras_worker_ForkPoolWorker-5:
Process Keras_worker_ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-6:
Process Keras_worker_ForkPoolWorker-1:
Process Keras_worker_ForkPoolWorker-2:
Process Keras_worker_ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/m

KeyboardInterrupt: ignored

#Predict

In [0]:
def image_file_to_json(img_path):
    img_dir = os.path.dirname(img_path)
    img_id = os.path.basename(img_path).split('.')[0]

    return img_dir, [{'image_id': img_id}]


def image_dir_to_json(img_dir, img_type='jpg'):
    img_paths = glob.glob(os.path.join(img_dir, '*.'+img_type))

    samples = []
    for img_path in img_paths:
        img_id = os.path.basename(img_path).split('.')[0]
        samples.append({'image_id': img_id})

    return samples


def predict(base_model_name, weights_file, image_source, predictions_file, img_format='jpg'):
    # load samples
    if os.path.isfile(image_source):
        image_dir, samples = image_file_to_json(image_source)
    else:
        image_dir = image_source
        samples = image_dir_to_json(image_dir, img_type='jpg')

    # build model and load weights
    nima = Nima(base_model_name, weights=None)
    nima.build()
    nima.nima_model.load_weights(weights_file)

    # initialize data generator
    data_generator = TestDataGenerator(samples, image_dir, 64, 10, nima.preprocessing_function(),
                                       img_format=img_format)

    # get predictions
    predictions = nima.nima_model.predict_generator(data_generator, workers=8, use_multiprocessing=True, verbose=1)

    # calc mean scores and add to samples
    for i, sample in enumerate(samples):
        sample['mean_score_prediction'] = calc_mean_score(predictions[i])

    print(json.dumps(samples, indent=2))

    if predictions_file is not None:
        save_json(samples, predictions_file)


#Main for predict

In [15]:
BASE_MODEL_NAME = "MobileNet"
WEIGHTS_FILE = '/content/gdrive/My Drive/GC-IQA_Prism/IQA_Prism/weights/E1/weights_mobilenet_50_0.074.hdf5'
IMAGE_SOURCE = '/content/gdrive/My Drive/GC-IQA_Prism/IQA_Prism/test_images'
PREDICTIONS_FILE = '/content/gdrive/My Drive/GC-IQA_Prism/IQA_Prism/predictions/E1.json'

ensure_file_exists(PREDICTIONS_FILE)

predict(base_model_name=BASE_MODEL_NAME, weights_file=WEIGHTS_FILE, image_source=IMAGE_SOURCE, predictions_file=PREDICTIONS_FILE)

Instructions for updating:
Please use Model.predict, which supports generators.
1/1 [==============================] - 0s 64ms/step
[
  {
    "image_id": "42041",
    "mean_score_prediction": 4.103508848696947
  },
  {
    "image_id": "42044",
    "mean_score_prediction": 3.919870112091303
  },
  {
    "image_id": "42042",
    "mean_score_prediction": 4.849147483706474
  },
  {
    "image_id": "42040",
    "mean_score_prediction": 3.5831059627234936
  },
  {
    "image_id": "42039",
    "mean_score_prediction": 3.843812245875597
  }
]
